In [1]:
cd notebooks/

/home/nicit/deep_learning_course_wfco/notebooks


In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

In [3]:
folder_path = "/home/nicit/nordsoen/nordsoen-i-wflo/results/DL/recorders/"

recorders = {}
n_layouts = 10000
# load pickle
for random_state in range(int(n_layouts/100)):
    with open(folder_path + f"recorder_{random_state}.pkl", "rb") as f:
        recorders[random_state] = pickle.load(f)

In [4]:
recorders[0][1000].keys()

dict_keys(['yaw', 'x', 'y', 'x_rot', 'y_rot', 'd_x', 'd_y', 'sort_idx', 'ws_eff', 'ti_eff', 'ws', 'aep', 'aep_opt'])

In [5]:
print('d_x shape: ', recorders[0][1000]['d_x'].shape)
print('d_y shape: ', recorders[0][1000]['d_y'].shape)
print('yaw shape: ', recorders[0][1000]['yaw'].shape)
print('ws_eff shape: ', recorders[0][1000]['ws_eff'].shape)
print('ti_eff shape: ', recorders[0][1000]['ti_eff'].shape)
print('ws shape: ', recorders[0][1000]['ws'].shape)
print('aep shape: ', recorders[0][1000]['aep'])
print('aep_opt shape: ', recorders[0][1000]['aep_opt'].shape)

d_x shape:  (360, 46)
d_y shape:  (360, 46)
yaw shape:  (46, 360, 11)
ws_eff shape:  (46, 360, 11)
ti_eff shape:  (46, 360, 11)
ws shape:  (11,)
aep shape:  [[0.00437259 0.00587007 0.03076611 ... 0.13747569 0.01684455 0.02444979]
 [0.00434477 0.00583548 0.03062883 ... 0.14218515 0.01743774 0.02528761]
 [0.00431171 0.00579388 0.03045422 ... 0.14669767 0.01801246 0.02614194]
 ...
 [0.00448134 0.00601999 0.0316177  ... 0.17683724 0.02217008 0.03242694]
 [0.00445364 0.00598145 0.03139479 ... 0.16316903 0.02030696 0.02962891]
 [0.00441877 0.00593384 0.03112336 ... 0.15002525 0.01852943 0.02696669]]
aep_opt shape:  (360, 11)


In [6]:
def make_case_tensors(result):
    d_x = result["d_x"]           # (360, 46)
    d_y = result["d_y"]           # (360, 46)
    ws_eff = result["ws_eff"]     # (46, 360, 11)
    ti_eff = result["ti_eff"]     # (46, 360, 11)
    ws = result["ws"]             # (11,)
    # aep = result["aep"]           # (360, 11)
    yaw_opt = result["yaw"]       # (46, 360, 11)  

    n_dir, n_wt = d_x.shape
    n_ws = len(ws)
    n_cases = n_dir * n_ws

    # --- Flatten and align all inputs ---
    # Repeat static layout (d_x, d_y) across wind speeds
    d_x_flat = np.repeat(d_x.T[:, :, None], n_ws, axis=2).reshape(n_wt, n_cases)
    d_y_flat = np.repeat(d_y.T[:, :, None], n_ws, axis=2).reshape(n_wt, n_cases)

    # Flatten dynamic inputs
    ws_eff_flat = ws_eff.reshape(n_wt, n_cases)
    ti_eff_flat = ti_eff.reshape(n_wt, n_cases)

    # Combine features per turbine (turbine-wise ordering)
    X_grouped = np.stack([d_x_flat, d_y_flat, ws_eff_flat, ti_eff_flat], axis=1)
    X_turb = X_grouped.reshape(n_wt * 4, n_cases)  # (46*4, n_cases)

    # Append wind speed and AEP as global inputs
    ws_rep = np.tile(ws, n_dir).reshape(1, n_cases)  # (1, n_cases)
    # aep_flat = aep.reshape(1, n_cases)               # (1, n_cases)

    # Final input matrix
    X = np.vstack([X_turb, ws_rep])        # , aep_flat (46*4 + 2, n_cases)

    # --- Output: yaw_opt reshaped per turbine ---
    Y = yaw_opt.reshape(n_wt, n_cases)              # (46, n_cases)

    # Convert to torch tensors
    X_torch = torch.from_numpy(X).half()
    Y_torch = torch.from_numpy(Y).half()

    return X_torch, Y_torch


# === Build full dataset ===
X_list, Y_list = [], []

for recorder_id, results in tqdm(recorders.items(), desc="Processing recorders:", colour="green"):
    for result_id, result in results.items():
        try:
            X, Y = make_case_tensors(result)  # Y now contains yaw_opt (46 x n_cases)
            X_list.append(X)
            Y_list.append(Y)
        except Exception as e:
            print(f"Error on {recorder_id}-{result_id}: {e}")

# --- Save layout-wise dataset to file ---
torch.save({'X': torch.cat(X_list, dim=1), 'Y': torch.cat(Y_list, dim=1)}, f'../data/dataset_yaw_{n_layouts}.pt')
print(f"✅ Saved {len(X_list)} layouts to f'dataset_yaw_{n_layouts}.pt'")


Processing recorders:: 100%|██████████| 100/100 [09:03<00:00,  5.43s/it]


✅ Saved 10000 layouts to 'dataset_yaw_10000.pt'
